In [24]:
import pandas as pd 
import xarray as xr
import matplotlib.pyplot as plt 
import numpy as np
import os
import seaborn as sns

# import xycmap

import functions


In [25]:
tl_folder = f'/Users/varyabazilova/Desktop/paper2/202406_modelruns/30years/2024Nov_output/TL/'


In [33]:
%%time

# Define the list of landcover indices and percentiles
landcover_indices = [1, 2, 3, 4]
# percentiles = ['20percent', '30percent', '40percent', '50percent', '60percent']
percentiles = ['50percent']#, '60percent']
method = 'once'  # Define your method here
column = 'Qstl'  # Set the target column
location = 'langtang'
folder_langtang = '12a'
folder_mustang = '13a'


# Load elevation data and define the output path
elevation = pd.read_csv(f'/Users/varyabazilova/Desktop/paper2/downscaling_simple/coordinates_and_elevation_with_labels_{location}.csv')[['cellnr2', 'band_data']]
elevation = elevation.transpose()
elevation_list = elevation.loc['cellnr2'].tolist()  # Adjust if you need a specific list format




# Loop over each landcover index
for landcover_idx in landcover_indices:
    # Loop over each percentile
    for percentile in percentiles:
        print(f"Processing landcover index {landcover_idx} and percentile {percentile}")

        # output_folder = f'/Users/varyabazilova/Desktop/paper2/202406_modelruns/30years/2024Nov_output/SL_{method}/output_{percentile}'
        output_folder = f'/Users/varyabazilova/Desktop/paper2/202406_modelruns/30years/2024Nov_output/TL'
        # Define folder paths for each iteration
        # folder_path = f'/Volumes/Extreme SSD/202409_paper2_modelruns/30years/SL_{method}/{landcover_idx}landcover_{percentile}/{location}_climate_cut'
        folder_path = f'/Users/varyabazilova/Desktop/paper2/202406_modelruns/30years/SL_data_once/{landcover_idx}landcover_{percentile}/{location}_climate_cut'

        # timestep workaround 
        df = functions.calculate_monthly_sediment_yield_all(pd.read_csv(folder_path + f'/{folder_langtang}/Sediment.out'), column)
        df = df[['month', 'year']]

        
        result_df = pd.DataFrame()

        # Iterate over files in the folder
        for folder_name in os.listdir(folder_path):
            # Skip unwanted files
            if folder_name.endswith('.csv') or folder_name.startswith('.'):
                continue

            file_path = os.path.join(folder_path, folder_name, 'Sediment.out')
            if os.path.isfile(file_path):
                # Read sediment data and calculate mean monthly value
                sediments = pd.read_csv(file_path)
                output_df = functions.calculate_monthly_sediment_yield_all(sediments, column)
                
                # Take the column and rename it
                name_column = output_df[column]
                column_name = f'{column}_{folder_name}'
                result_df[column_name] = name_column
                result_df.columns = [col[-3:] for col in result_df.columns]  # Rename columns for each folder


        
        # # Filter result_df by elevation list
        result_df = result_df[elevation_list]

        # Check for consistency with elevation list
        if elevation_list == result_df.columns.tolist():
            print("default land cover. monthly data: same")
            
            # Rename columns, merge with timestep, and add land cover info
            result_df.columns = elevation.loc['band_data']
            result_df = pd.concat([df, result_df], axis=1)  # Assuming `df` is defined elsewhere and aligns as expected
            result_df['land_cover'] = f'landcover{landcover_idx}'

            # Define the output file name and save the DataFrame to a CSV file
            output_filename = f'{location}_monthly_sum_elevation_{column}_landcover{landcover_idx}_{percentile}.csv'
            result_df.to_csv(os.path.join(output_folder, output_filename), index=False)

            print(f"Saved {output_filename}")
        else: print('its not the same AAAA!')

Processing landcover index 1 and percentile 50percent
default land cover. monthly data: same
Saved langtang_monthly_sum_elevation_Qstl_landcover1_50percent.csv
Processing landcover index 2 and percentile 50percent
default land cover. monthly data: same
Saved langtang_monthly_sum_elevation_Qstl_landcover2_50percent.csv
Processing landcover index 3 and percentile 50percent
default land cover. monthly data: same
Saved langtang_monthly_sum_elevation_Qstl_landcover3_50percent.csv
Processing landcover index 4 and percentile 50percent
default land cover. monthly data: same
Saved langtang_monthly_sum_elevation_Qstl_landcover4_50percent.csv
CPU times: user 1min 5s, sys: 2.18 s, total: 1min 7s
Wall time: 1min 8s


In [17]:
result_df

,month,year,3689.0,2546.0,3964.0,2571.0,2886.0,3795.0,3540.0,4420.0,...,4785.0,4439.0,4593.0,4603.0,4733.0,4905.0,4952.0,4812.0,4889.0,land_cover
0,9,1989,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,landcover4
1,10,1989,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,landcover4
2,11,1989,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,landcover4
3,12,1989,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,landcover4
4,1,1990,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,landcover4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
392,5,2022,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,landcover4
393,6,2022,35051.613137,34985.205771,35067.139108,34986.670068,16696.428324,16696.428324,16696.428324,16696.428324,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,landcover4
394,7,2022,16388.839363,3555.042302,11107.602630,3573.740813,36148.272563,31463.138577,31386.035923,29984.658567,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,landcover4
395,8,2022,8191.154857,6136.982177,7352.665882,6140.192816,8414.761288,7668.720844,7656.443351,7433.294091,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,landcover4


In [ ]:
result_df.columns = elevation.loc['band_data']

In [ ]:
result_df

In [ ]:
%%time

# Define the list of landcover indices and percentiles
landcover_indices = [1, 2, 3, 4]
percentiles = ['20percent', '30percent', '40percent', '50percent', '60percent']
location = 'langtang'
method = 'daily' 

# Set the column and other parameters
column = 'Q100'

df = pd.DataFrame()  # Assuming `df` is initialized as needed

# Loop over each landcover index
for landcover_idx in landcover_indices:
    # Loop over each percentile
    for percentile in percentiles:
        print(f"Processing landcover index {landcover_idx} and percentile {percentile}")
        
        # Define folder paths
        folder_path = f'/Volumes/Extreme SSD/202409_paper2_modelruns/30years/SL_{method}/{landcover_idx}landcover_{percentile}/{location}_climate_cut'
        output_folder = f'/Users/varyabazilova/Desktop/paper2/202406_modelruns/30years/2024Nov_output/SL_{method}/output_{percentile}/'
        result_df = pd.DataFrame()

        # Iterate over files in the folder
        for folder_name in os.listdir(folder_path):
            # Skip hidden and .csv files
            if folder_name.endswith('.csv') or folder_name.startswith('.'):
                continue

            file_path = os.path.join(folder_path, folder_name, 'Sediment.out')
            if os.path.isfile(file_path):
                # Read sediment data and calculate monthly sediment yield
                sediments = pd.read_csv(file_path)
                output_df = functions.calculate_monthly_sediment_yield_all(sediments, column)
                
                # Take the specified column and rename it
                name_column = output_df[column]
                column_name = f'{column}_{folder_name}'
                result_df[column_name] = name_column
                result_df.columns = [col[-3:] for col in result_df.columns]

        # Filter and check columns with elevation list
        result_df = result_df[elevation_list]
        if elevation_list == result_df.columns.tolist():
            print("default land cover. monthly data: same")
        else:
            print("not the same")

        # Rename columns according to elevation, merge with timestep, and add land cover info
        result_df.columns = elevation.loc['band_data']
        result_df = pd.concat([df, result_df], axis=1)
        result_df['land_cover'] = f'landcover{landcover_idx}'

        # Save the DataFrame to a CSV file with both landcover and percentile in the filename
        output_filename = f'{location}_monthly_sum_elevation_{column}_landcover{landcover_idx}_{percentile}.csv'
        result_df.to_csv(os.path.join(output_folder, output_filename), index=False)

        print(f"Saved {output_filename}")
